# Microgrid optimization with an algebraic approach using JuMP


An experiment which derives from the **Microgrid sizing optimization** notebook example in `../Microgrids.jl/examples/`. However, instead of doing a sizing optimization using a blackbox approach which calls the Microgrid simulator, here we use an algebraic description of the problem (using JuMP) and we run an "all-in-one" optimization of both the sizing and the energy flows at each instant.

Caveat: this yields an anticipative energy management.

PH, oct 2023

In [1]:
using Microgrids
using JuMP
using HiGHS # Linear Programming solver
using Printf # pretty print results
#using Random, Statistics
#using CSV, DataFrames
using PyPlot

[ Info: Precompiling Microgrids [bd581358-d3fa-499e-a26e-e70307242c03]


## Load Microgrid project data

Loading parameters and time series for a Microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*. 
Values gathered from the Microgrid_Wind-Solar.ipynb example notebook.

In [3]:
include("../../Microgrids.jl/examples/Microgrid_Wind-Solar_data.jl");

Data definition for Microgrid with wind, solar, storage and generator...


In [34]:
"""Create a Microgrid project description of size `x`
with x=[power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind] in kW.

if x is omitted, x=[1,1,...] is used
"""
function create_microgrid(x)
    project = Project(lifetime, discount_rate, timestep, "€")
    # Split decision variables (converted MW → kW):
    power_rated_gen = x[1]
    energy_rated_sto = x[2]
    power_rated_pv = x[3]
    power_rated_wind = x[4]
    # Create components
    gen = DispatchableGenerator(power_rated_gen,
        fuel_intercept, fuel_slope, fuel_price,
        investment_price_gen, om_price_gen, lifetime_gen,
        load_ratio_min,
        replacement_price_ratio, salvage_price_ratio, fuel_unit)
    batt = Battery(energy_rated_sto,
        investment_price_sto, om_price_sto, lifetime_sto, lifetime_cycles,
        charge_rate, discharge_rate, loss_factor_sto, SoC_min, SoC_ini,
        replacement_price_ratio, salvage_price_ratio)
    pv = Photovoltaic(power_rated_pv, irradiance,
        investment_price_pv, om_price_pv,
        lifetime_pv, derating_factor_pv,
        replacement_price_ratio, salvage_price_ratio)
    windgen = WindPower(power_rated_wind, cf_wind,
        investment_price_wind, om_price_wind,
        lifetime_wind,
        replacement_price_ratio, salvage_price_ratio)
    mg = Microgrid(project, Pload, gen, batt, [pv, windgen])

    return mg
end

function create_microgrid()
    x1 = [1., 1., 1., 1.]
    return create_microgrid(x1)
end

create_microgrid (generic function with 2 methods)

max bounds

In [35]:
Pload_max = maximum(Pload)
power_rated_gen_max = 1.2 * Pload_max
energy_rated_sto_max = 10.0 * Pload_max
power_rated_pv_max = 10.0 * Pload_max
power_rated_wind_max = 5.0 * Pload_max

8535.0

Create Microgrid project description. Since we'll use it for zizing, it's fine if the size is set to an bad sizing.

In [89]:
mg = create_microgrid();
K = length(mg.load)
dt = mg.project.timestep
#traj, stats, costs = simulate(mg)
#stats

1.0

## JuMP model

In [144]:
model = Model(HiGHS.Optimizer)
set_silent(model)

### sizing variables

In [145]:
@variable(model, 0 <= power_rated_gen <= power_rated_gen_max)
@variable(model, 0 <= energy_rated_sto <= energy_rated_sto_max)
@variable(model, 0 <= power_rated_pv <= power_rated_pv_max)
@variable(model, 0 <= power_rated_wind <= power_rated_wind_max)

power_rated_wind

### Non dispatchable sources

In [146]:
@variable(model, pv_potential[1:K])
@constraint(model, pv_potential .== power_rated_pv*production(mg.nondispatchables[1]))
@variable(model, wind_potential[1:K])
@constraint(model, wind_potential .== power_rated_wind*production(mg.nondispatchables[2]));
# @variable(model, renew_potential[1:K])
# @constraint(model, renew_potential .== pv_potential .+ wind_potential);

### Net load (desired, i.e. before spillage of excess renewables and load shedding)

In [147]:
@variable(model, Pnl[1:K])
@constraint(model, Pnl .== mg.load .- pv_potential .+ wind_potential);

In [148]:
@variable(model, Pspill[1:K] >= 0)
@variable(model, Pshed[1:K] >= 0);

### Dispatchable generator

In [149]:
@variable(model, Pgen[1:K] >= 0)
@constraint(model, Pgen .<= power_rated_gen);

### Energy storage

In [150]:
@variable(model, Psto_cha[1:K])
@variable(model, Psto_dis[1:K])
@variable(model, Esto[1:K+1] >= 0) # replace with mg.storage.SoC_min
@constraint(model, Esto .<= energy_rated_sto);
# TODO: add power constraints
#@constraint(model, ... <= Psto .<= ...);

Evolution of the State of Energy

TODO: add storage losses (PWL)

In [151]:
mg.storage.loss_factor

0.05

In [152]:
mg.storage.SoC_ini

0.0

In [153]:
@constraint(model, sto[k in 1:K], Esto[k+1] == Esto[k] + Psto_cha[k]*dt - Psto_dis[k]*dt)
sto[1]

sto[1] : -Psto_cha[1] + Psto_dis[1] - Esto[1] + Esto[2] = 0

Storage Cyclicity

In [154]:
@constraint(model, Esto[K+1] == Esto[1])

-Esto[1] + Esto[8761] = 0

### Power balance

In [155]:
@constraint(model, balance, Pnl .== Pgen + Psto_dis - Psto_cha + Pshed - Pspill)
balance[1]

balance : Pnl[1] + Pspill[1] - Pshed[1] - Pgen[1] + Psto_cha[1] - Psto_dis[1] = 0

Shedding limit:

In [156]:
fix.(Pshed, 0.0; force=true);

### Cost

🚧 **TO BE CONTINUED....** 🚧

In [157]:
NPC = power_rated_gen + energy_rated_sto +  power_rated_pv + power_rated_wind

power_rated_gen + energy_rated_sto + power_rated_pv + power_rated_wind

In [158]:
@objective(model, Min, NPC)

power_rated_gen + energy_rated_sto + power_rated_pv + power_rated_wind

In [159]:
optimize!(model)

In [160]:
value(power_rated_gen), value(energy_rated_sto), value(power_rated_pv), value(power_rated_wind)

(1607.0, 100.0, 0.0, 0.0)

---

In [140]:
model

A JuMP Model
Minimization problem with:
Variables: 78845
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 35041 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 17521 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 8760 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 26285 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: Esto, Pgen, Pnl, Pshed, Pspill, Psto_cha, Psto_dis, energy_rated_sto, power_rated_gen, power_rated_pv, power_rated_wind, pv_potential, sto, wind_potential

In [26]:
#print(model)

In [ ]:
renew_productions

---

## Previous notebook content

In [4]:
"Multi-objective criterion for microgrid performance: lcoe, shedding rate"
function obj_multi(x)
    stats, costs = simulate_microgrid(x)
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate; # in [0,1]
    return lcoe, shed_rate
end

obj_multi

In [5]:
"""Mono-objective criterion: LCOE + penalty if shedding rate > `shed_max`

with signature adapted to NLopt with `grad` as 2nd argument

load shedding penalty threshold `shed_max` should be in  [0,1[
"""
function obj(x, grad, shed_max, w_shed_max=1e5)
    lcoe, shed_rate = obj_multi(x)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0
        penalty = w_shed_max*over_shed
    else
        penalty = 0.0
    end
    J = lcoe + penalty
end

obj

### Tests the objective functions

Test of the simulator wrapper (on a baseline sizing):

In [6]:
# Baseline sizing: same as in Microgrid_Wind-Solar.ipynb notebook
x_base = [power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]/1000.
# run simulation:
stats, costs = simulate_microgrid(x_base)
x_base, costs.lcoe, costs.npc/1e6

([1.8, 5.0, 3.0, 0.9], 0.22924812869928668, 21.89002772908652)

In [7]:
# Test:
shed_max = 0.01 # in [0,1]

println("Base. multi: ", obj_multi(x_base), " mono: ", obj(x_base,[], shed_max))

Base. multi: (0.22924812869928668, 0.0) mono: 0.22924812869928668


## Optimization

### Setting up the optimization problem

bounds of the design space and starting point: derived from maximal load power

In [8]:
Pload_max = maximum(Pload)

xmin = [0., 0., 1e-3, 0.] # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
xmax = [1.2, 10.0, 10.0, 5.0] * (Pload_max/1000)

4-element Vector{Float64}:
  2.0484
 17.07
 17.07
  8.535

Optionally ban some choices:

In [9]:
# Solar power forbidden (optional)
#xmax[3] = 1e-3
# Wind power forbidden (optional)
#xmax[4] = 0.

### Wrapper of the optimization process

This is an optional step, but recommended to explore easily the impact of the many parameters taken by optimization algorithms.

See [optimization termination conditions](https://nlopt.readthedocs.io/en/latest/NLopt_Introduction/#termination-conditions) in NLopt documention for the meaning of `xtol_rel`

In [12]:
"""Optimize sizing of microgrid based on the `obj` function

Parameters:
- `x0`: initial sizing (for the algorithms which need them)
- `shed_max`: load shedding penalty threshold (same as in `obj`)
- `algo` could be one of LN_SBPLX, GN_DIRECT, GN_ESCH...
- `maxeval`: maximum allowed number of calls to the objective function,
  that is to the microgrid simulation
- `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
- `srand`: random number generation seed (for algorithms which use some stochastic search)

Problem bounds are taken as the global variables `xmin`, `xmax`,
but could be added to the parameters as well.
"""
function optim_mg(x0, shed_max, algo=:LN_SBPLX, maxeval=1000, xtol_rel=1e-4, srand=1)
    nx = length(x0) # number of optim variables
    opt = Opt(algo, nx)
    NLopt.srand(srand)
    
    opt.lower_bounds = xmin
    opt.upper_bounds = xmax
    opt.min_objective = (x, grad) -> obj(x, grad, shed_max)
    opt.xtol_rel = xtol_rel
    opt.maxeval = maxeval
    
    (fopt, xopt, ret) = optimize(opt, x0)
    return fopt, xopt, ret, opt.numevals
end

optim_mg